In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error, ndcg_score, dcg_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from select_data import select#, intersection



In [2]:
df=pd.read_csv('matrice_avis_utilisateur.csv')

In [3]:
from select_data import select

data=select(df)

             movie       rating
count  3706.000000  3706.000000
mean   1995.573125   269.889099
std    1151.148045   384.047838
min       1.000000     1.000000
25%     989.250000    33.000000
50%    2033.500000   123.500000
75%    2990.750000   350.000000
max    3952.000000  3428.000000
            rating         user
count  6040.000000  6040.000000
mean    130.904305  3020.500000
std     136.600298  1743.742145
min       7.000000     1.000000
25%      37.000000  1510.750000
50%      80.000000  3020.500000
75%     171.000000  4530.250000
max     980.000000  6040.000000


In [4]:
# selection des users qui ont un std large

In [5]:
# split des data
X_train, X_test = train_test_split(data, train_size=0.8, random_state=42, shuffle=True)
len(data),len(X_train),len(X_test)

(43654, 34923, 8731)

In [6]:
# conservation des films qui ont eté vue a l'entrainement
X_test=X_test[X_test['movie'].isin(X_train['movie'].unique())]
len(X_test)

8731

In [7]:
# conservation des users qui ont eté vue a l'entrainement
X_test=X_test[X_test['user'].isin(X_train['user'].unique())]
len(X_test)

8731

In [8]:
train_sparse = X_train.astype(pd.SparseDtype("float", 0))
test_sparse = X_test.astype(pd.SparseDtype("float", 0))

In [9]:
dtrain=train_sparse.pivot(columns='movie',index='user',values='rating').fillna(0)
dtest=test_sparse.pivot(columns='movie',index='user',values='rating').fillna(0)

In [10]:
# with mlflow.start_run() as run:
#     mlflow.sklearn.log_model(NMFRecommender, "NMFRecommender", registered_model_name="NMFRecommenderModel")
#     mlflow.log_params({"n_components": n_components, "max_iter": max_iter})
#     mlflow.log_metric("Training MSE", mse_train)
#     mlflow.log_metric("Test MSE", mse_test)

In [11]:
import mlflow

n_components=18
max_iter=2000

mlflow.autolog()

with mlflow.start_run():
    # mlflow.sklearn.log_model(NMFRecommender, "NMFRecommender", registered_model_name="NMFRecommenderModel")
    mlflow.log_params({"n_components": n_components, "max_iter": max_iter})

    nmf = NMF(n_components=n_components,max_iter=max_iter)
    
    
   # Fit the model to the user-item matrix
    W = nmf.fit_transform(dtrain.values)  # User matrix
    H = nmf.components_  # Item matrix

    # nmf_result_test = nmf.transform(dtest.values)

    pred_matrix = np.dot(W,H)


2024/02/16 15:03:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/02/16 15:03:13 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/02/16 15:03:13 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/02/16 15:03:18 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Utilisateur\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [12]:
nmf = NMF(n_components=100,max_iter=10000)
# Fit the model to the user-item matrix
W = nmf.fit_transform(dtrain.values)  # User matrix
H = nmf.components_  # Item matrix
pred_matrix = np.dot(W,H) 

2024/02/16 15:03:18 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a509ffe5167441e3bdd28085e8d0cf50', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/02/16 15:03:23 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/02/16 15:03:23 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [13]:
scoring=pd.DataFrame(pred_matrix)
scoring.columns=dtrain.columns
scoring.index=dtrain.index

In [14]:
# from sklearn.preprocessing import MinMaxScaler

# scoring.T
# scaler=MinMaxScaler()
# scaler.fit([[1,5],[1,5]])
# scaler.transform(scoring)
# print(scaler.data_max_)
# scoring=scaler.transform(scoring)





In [15]:
unpivot_scoring=scoring.reset_index().melt(id_vars='user', var_name='movie', value_name='scoring')

In [16]:
unpivot_scoring['user']=unpivot_scoring['user'].sparse.to_dense()

In [17]:
train_pred_df = pd.merge(X_train, unpivot_scoring, on=['user', 'movie'])
test_pred_df = pd.merge(X_test, unpivot_scoring, on=['user', 'movie'])

In [18]:
print(mean_squared_error(train_pred_df['rating'],train_pred_df['scoring']))
print(mean_squared_error(test_pred_df['rating'],test_pred_df['scoring']))

2.8407900892099804
12.65466309650533


In [19]:
df


,movie,user,timestamp,rating
0,858,6040,956703932,4
1,2384,6040,956703954,4
2,593,6040,956703954,5
3,2019,6040,956703977,5
4,1961,6040,956703977,4
...,...,...,...,...
1000204,2399,4958,1046454338,1
1000205,1407,4958,1046454443,5
1000206,3264,4958,1046454548,4
1000207,2634,4958,1046454548,3


In [20]:
# intersection proportion

def intersection(value):
    total=0
    total_user_section_size=0
    list_accuracy=[]

    for indice,user in enumerate(train_pred_df['user'].unique()):

        # if indice>0 :
        #     break
        
        user_section=test_pred_df[test_pred_df['user']==user]
        user_section_size=len(user_section)
        size_top=value/100
        limit=round(user_section_size*size_top)
        total_user_section_size+=user_section_size


        user_pref=user_section.sort_values('rating',ascending=False)['movie'][:limit]
        # print(len(user_pref))
        # print((user_pref))
        user_recommandation=user_section.sort_values('scoring',ascending=False)['movie'][:limit]
        # (len(user_recommandation))
        # print((user_recommandation))
        valid_recommandation=user_recommandation[user_recommandation.isin(user_pref)]

        how_many_valid_recomandation=len(valid_recommandation)
        try : 
            accuracy=(how_many_valid_recomandation/user_section_size)*100
        except : 
            accuracy=0


        list_accuracy+=[accuracy]

        total+=how_many_valid_recomandation

    mean_intersection=round(total/len(train_pred_df['user'].unique()))
    mean_user_section_size=round(total_user_section_size/len(train_pred_df['user'].unique()))
    mean_accuracy=(mean_intersection/mean_user_section_size)*100

    std=(1/2)**sum([ (x-mean_accuracy)**2 for x in list_accuracy])
    # print(mean_intersection,mean_user_section_size)
    return mean_accuracy,std




In [21]:
for x in range(10,101,10):
    mean,std=intersection(x)
    print(x,'%','accuracy',mean,std,sep=' ')

10 % accuracy 0.0 0.0
20 % accuracy 3.8461538461538463 0.0
30 % accuracy 11.538461538461538 0.0
40 % accuracy 19.230769230769234 0.0
50 % accuracy 26.923076923076923 0.0
60 % accuracy 38.46153846153847 0.0
70 % accuracy 50.0 0.0
80 % accuracy 65.38461538461539 0.0
90 % accuracy 80.76923076923077 0.0
100 % accuracy 100.0 1.0


In [22]:
for x in range(10,101,10):
    mean,std=intersection(x)
    print((mean/x)*100)
    

0.0
19.230769230769234
38.46153846153847
48.07692307692308
53.84615384615385
64.1025641025641
71.42857142857143
81.73076923076923
89.74358974358975
100.0


In [23]:
mlflow.log_metric("Training MSE",mean_squared_error(train_pred_df['rating'],train_pred_df['scoring']))
                      
mlflow.log_metric("Test MSE",mean_squared_error(test_pred_df['rating'],test_pred_df['scoring']))


In [24]:
print(ndcg_score([train_pred_df['rating'].values], [train_pred_df['scoring'].values]))
print(ndcg_score([test_pred_df['rating'].values], [test_pred_df['scoring'].values]))

0.9932303500859223
0.9748281215705205


In [25]:
print(dcg_score([train_pred_df['rating'].values], [train_pred_df['scoring'].values]))
print(dcg_score([test_pred_df['scoring'].values], [test_pred_df['scoring'].values]))

9847.17960759842
356.9799176416209


In [26]:
def run (file): 
    df = pd.read_csv(csv_path)
    train_data, test_data, train_mini, test_mini=partition(df, test_size=0.2, mini_size=0.03):
    MF_training(n_components, max_iter, df):


SyntaxError: invalid syntax (3144673885.py, line 3)